In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2   #opencv
import os
import sys
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *
from PIL import Image
import wx
import wx.lib.filebrowsebutton

%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=sys.maxsize)

In [ ]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [ ]:
print("Total Params:", FRmodel.count_params())

In [ ]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss as defined by formula
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]

    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis = -1)

    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis = -1)

    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)

    loss = tf.reduce_sum(tf.maximum(basic_loss, 0))

    
    return loss

In [ ]:
with tf.Session() as test:
    tf.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random_normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random_normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random_normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)
    
    print("loss = " + str(loss.eval()))

In [ ]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

In [ ]:
database = {}
database["Neha"] = img_to_encoding("images/crop/neha1.jpg", FRmodel)
database["Kalu"] = img_to_encoding("images/crop/kalu1.jpg", FRmodel)
database["Suds"] = img_to_encoding("images/crop/suds1.jpg", FRmodel)

In [ ]:
def who_is_it(image_path, database, model):
    """
    Implements face recognition by finding who is the person on the image_path image.
    
    Arguments:
    image_path -- path to an image
    database -- database containing image encodings along with the name of the person on the image
    model -- your Inception model instance in Keras
    
    Returns:
    min_dist -- the minimum distance between image_path encoding and the encodings from the database
    identity -- string, the name prediction for the person on image_path
    """

    
    encoding = img_to_encoding(image_path, model)
    min_dist = 100
    for (name, db_enc) in database.items():
        dist = np.linalg.norm(encoding - db_enc)
        if dist < min_dist:
            min_dist = dist
            identity = name

    if min_dist > 0.65:
        return("Not in the database.")
    else:
        return("It's " + str(identity)) #+ ", the distance is " + str(min_dist))

        
  #  return min_dist, identity

In [ ]:
who_is_it("images/crop/neha2.jpg", database, FRmodel)

In [ ]:
for key,value in database.items() :
    print (key)

In [ ]:
# Crop Images
!autocrop -i images\pics -o images\crop -r images\reject -H 96 -w 96

In [ ]:
class MyFrame(wx.Frame):    
        
    def on_show(self, event):
        path = str(self.fbb.GetValue())
        img  = Image.open(path)
        img.show()
        
    def get_list(self):
        text = "Verified List: "
        for key,value in database.items() :
            text += str(key) + ", "
        return text
    
    def on_add(self, event):
        path = str(self.addtodict.GetValue())
        dlg = wx.TextEntryDialog(self.panel, 'Enter Name:',"Add New Entry","", 
                style=wx.OK)
        dlg.ShowModal()
        name_val = str(dlg.GetValue())
        database[name_val] = img_to_encoding(path, FRmodel)
        text = self.get_list()
        self.label_list.SetLabel(text)
        
    def on_perform(self, event):
        path = str(self.fbb.GetValue())
        output = who_is_it(path, database, FRmodel)
        wx.MessageBox(output, "Result" ,wx.OK | wx.ICON_INFORMATION)
            
    def __init__(self):
        super().__init__(parent=None, title='Face Recognition', size = (600,480))
        self.panel = wx.Panel(self)        
        self.Bind(wx.EVT_CLOSE, self.OnCloseWindow)
        my_sizer = wx.BoxSizer(wx.VERTICAL)    
        
        self.addtodict = wx.lib.filebrowsebutton.FileBrowseButton(self.panel,
            labelText="Select new Entry (Should be 96x96 img):", fileMask="*.jpg")
        my_sizer.Add(self.addtodict, 0, wx.ALL | wx.CENTER, 10)
        
        btn1 = wx.Button(self.panel, label='Add to Database')
        btn1.Bind(wx.EVT_BUTTON, self.on_add)
        my_sizer.Add(btn1, 0, wx.ALL | wx.CENTER, 5) 
        
        self.label_list = wx.StaticText(self.panel,-1,style = wx.ALIGN_CENTER)
        text = self.get_list()
        
        self.label_list.SetLabel(text)
        my_sizer.Add(self.label_list, 0, wx.ALL | wx.EXPAND, 50)
        
        self.fbb = wx.lib.filebrowsebutton.FileBrowseButton(self.panel,
            labelText="Select image to test:", fileMask="*.jpg")
        my_sizer.Add(self.fbb, 0, wx.ALL | wx.CENTER, 10)
        
        label_drop = wx.StaticText(self.panel,-1,style = wx.ALIGN_CENTER)
        text = "Available Operations"
        label_drop.SetLabel(text)
        my_sizer.Add(label_drop, 0, wx.ALL | wx.EXPAND, 10)
        
        
        btn2 = wx.Button(self.panel, label='Show Image')
        btn2.Bind(wx.EVT_BUTTON, self.on_show)
        my_sizer.Add(btn2, 0, wx.ALL | wx.CENTER, 5) 
        
        btn3 = wx.Button(self.panel, label='Perform Test')
        btn3.Bind(wx.EVT_BUTTON, self.on_perform)
        my_sizer.Add(btn3, 0, wx.ALL | wx.CENTER, 5) 
        
        self.label_output = wx.StaticText(self.panel,-1,style = wx.ALIGN_CENTER)
        my_sizer.Add(self.label_output, 0, wx.ALL | wx.EXPAND, 10)
        
        self.panel.SetSizer(my_sizer)        
        self.Show()
        
    def OnCloseWindow(self, e):
        self.Destroy()

In [ ]:
app = wx.App()
frame = MyFrame()
app.MainLoop()

In [ ]:
# Run this Cell when "wx.App object must be created first!" Error Appears
del app